In [1]:
# 필요한 모듈 import

import pandas as pd
import requests #작은 브라우저로 웹사이트 정보 불러옴
from bs4 import BeautifulSoup as bs #html 태그 파싱
import random
import time
from tqdm import tqdm, trange #대량 데이터 처리 시, 시간 표시
import re

In [2]:
# url 가져오기

pnum = 1
year_month = 201906

base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300011&sugMonthTp={year_month}&orderKey=registDtDesc"

In [3]:
# content > div.suggestion_list > ul > li > dl > dt > p > a
# 연-월별 전체 제안 목록 불러오기

def get_suggestion_list(pnum):
    base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300011&orderKey=registDtDesc"
    response = requests.get( base_url )
    if response.status_code == 200: #OK
        html = bs(response.text, 'html.parser')
        tags = html.select('#content > div.suggestion_list > ul')[0].find_all('a')
        if not tags :
            return(suggestion_list)
        else :
            for tag in tags:
                suggestion_list.append(tag['href'])
        pnum += 1
        get_suggestion_list(pnum) #재귀호출
        print(pnum)
    else:
        return(suggestion_list)

In [4]:
# 목록의 내용만 크롤링
# suggestion_list: 목록의 링크 값

suggestion_list = []
pnum = 1

get_suggestion_list(pnum)
suggestion_list    

37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2


['/ucms/ogp/sug/view.do?menuNo=300011&sgId=217&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=216&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=215&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=214&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=210&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=209&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=208&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=207&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=206&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=205&pnum=2',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=204&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=203&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=202&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=201&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=200&pnum=3',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=199&pnum=4',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=198&pnum=4',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=197&p

In [5]:
# 게시물 내용 가져오기

url = '/ucms/ogp/sug/view.do?menuNo=300011&sgId=204&pnum=1'
base_url = f"https://www.innogov.go.kr{url}"
response = requests.get( base_url )
if response.status_code == 200:
    html = bs(response.text, 'html.parser')
    #content > div.vveiw_box1 > dl > dt
    title = html.select(
            '#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
    print(title)

문화 유적지(궁궐이나 왕릉 등)에  장애인을 위한 해설사를 배치


In [6]:
desc = html.select(
            '#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')
category = desc[0].get_text(strip=True)
print(desc[0].get_text(strip=True))
print(desc[1].get_text(strip=True))
print(desc[2].get_text(strip=True))
print(desc[3].get_text(strip=True))

인권/평등
2019.06.14
2019.09.11
ynkim97


In [7]:
# id값을 저장해 주기 위해 미리 추출 해보기

u = 'view.do?menuNo=300011&sgId=150&pnum=1'
u.split('=')[2].split('&')[0]

'150'

In [8]:
# 목록 리스트에 있는 url의 내용을 크롤링

def get_suggestion_content(url):
    article = []
    base_url = f"https://www.innogov.go.kr/{url}"
    response = requests.get( base_url )
    if response.status_code == 200:
        html = bs(response.text, 'html.parser')
        title = html.select(
            '#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
        desc = html.select(
            '#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')
        category = desc[0].get_text(strip=True)
        content = html.select(
            '#content > div.vveiw_box1 > div.vveiw_cont > div > pre')[0].get_text(strip=True)
        start = desc[1].get_text(strip=True)
        end = desc[2].get_text(strip=True)
        author = desc[3].get_text(strip=True)
        vote = html.select('#counter')[0].get_text(strip=True)
        sgId = url.split('=')[2].split('&')[0]
        
        article.append(sgId)
        article.append(title)
        article.append(category)
        article.append(content)
        article.append(start)
        article.append(end)
        article.append(vote)
        article.append(author)
        return article

In [9]:
# 각 게시물 세부내용 가져오기

data = []
# tqdm 을 사용해서 어느정도 가져왔는지 확인
for i, suggestion in tqdm(enumerate(suggestion_list)):
    # suggestion_list 에 있는 각 게시물에 접근해서 카테고리, 시작일, 마감일, 제안인, 제목, 내용 등을 가져온다
    s = get_suggestion_content(suggestion)
    data.append(s)

178it [00:47,  3.75it/s]


In [10]:
# 칼럼 이름 지정 후 데이터 프레임 만들기

column_names = ["sgId", "title", "category", "content", "start", "end", "vote", "author"]
data = pd.DataFrame(data, columns=column_names)
data.head()

,sgId,title,category,content,start,end,vote,author
0,217,근린공원,환경,포천시 근린공원 정책에 대해 정보제공과 주민간 의견수렴 과정이 있으면 좋겠습니다,2019.06.28,2019.09.25,0,저녁하늘
1,216,공시송달 의뢰공문의 전국배포를 중단하여 문서로 인한 스트레스 해소와 행정업무 효율성 증대,일반행정,"[현황 및 문제점]​0.행정절차법 제14조4항,지방세기본법 제33조2항에서는 공시송...",2019.06.27,2019.09.24,1,민첩지킴이
2,215,"중앙부처 및 산하기관에서 추진하는 공모사업, 지원 사업을 한곳에서 볼 수 있게 해 ...",국민참여,정부 각 부처에서는 훌륭한 공모사업이나 지원사업들을 많이 하고 있습니다.이런 내용을...,2019.06.24,2019.09.21,44,희동이
3,214,연가 기부제 도입,일반행정,아이 낳아 키우기 힘든 세상입니다.맞벌이 부부의 육아가 매우 힘듭니다.아이가 잦은 ...,2019.06.24,2019.09.21,1,탱크
4,210,어린이집교사 근무환경개선,환경,어린이집교사 처우 개선에 대해 개인적인 생각 ( 그냥 생각나는데로 써보겠습니다)저는...,2019.06.22,2019.09.19,3,꾸꾸바라기


In [11]:
# 카테고리별 글 개수 확인

data["category"].value_counts()

보건복지     29
일반행정     29
기타       26
안전       22
인권/평등    16
환경       12
상생협력     11
국민참여     10
반부패청렴     8
사회통합      8
공동체       7
Name: category, dtype: int64

In [12]:
# csv 파일로 저장

data.to_csv("./suggestion.csv", index=False)

In [13]:
# 데이터 크기 확인

data.shape

(178, 8)